In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df0 = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
df1 = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')

In [ ]:
df0.target = df0.target.map({val:i for i, val in enumerate(sorted(df0.target.unique()),1)})

In [ ]:
df0.target[:5]

## Manifold Learning / Dimension Reduction

In [ ]:
import sklearn
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, SpectralEmbedding, MDS
from sklearn.model_selection import train_test_split, StratifiedKFold

In [ ]:
sub_train = df0.iloc[:, 1:].sample(10000, random_state = 1)
sub_target = sub_train.target
sub_train_only = sub_train.iloc[:,:-1]
sub_train_only.shape, sub_target.shape

In [ ]:
from umap import UMAP

In [ ]:

umap = UMAP(random_state=1)
dr = umap.fit_transform(sub_train_only, sub_target)

In [ ]:
#fig, ax = plt.subplots(figsize = (10,10))
#sns.scatterplot(dr[:,0], dr[:,1], hue = sub_target , palette = sns.color_palette()[:4] )

In [ ]:
px.scatter(x=dr[:,0], y=dr[:,1], color = sub_target  )

In [ ]:
umap0 = UMAP(random_state=1)
dr0 = umap0.fit_transform(df0.iloc[:,1:-1], df0.target)

In [ ]:
px.scatter(x=dr0[:,0], y=dr0[:,1], color = df0.target  )

## UMAP data clustering as additional feature

Experiment it as addtional features or just using that clustering as feature for classificaiton. 
Experiment UMAP with different number of dimension output . 

Should try KNN classification with clustering results as input. 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report


In [ ]:
x, y = df0.iloc[:, 1:-1], df0.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 1, stratify = y)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
gbc = GradientBoostingClassifier(n_estimators = 300 , verbose = 1)


In [ ]:
gbc.fit(x_train, y_train)

In [ ]:
y_pred_gbc = gbc.predict(x_test)

In [ ]:
gbc.score(x_test, y_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred_gbc, labels = gbc.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = gbc.classes_)
disp.plot()

In [ ]:
print(classification_report(y_test, y_pred_gbc, labels = gbc.classes_))

In [ ]:
umap2 = UMAP(random_state=1)
x_train_umap_2 = umap2.fit_transform(x_train, y_train)
x_test_umap_2 = umap2.transform(x_test)

In [ ]:
gbc_umap_2 = GradientBoostingClassifier(n_estimators = 300 , verbose = 1)
gbc_umap_2.fit(x_train_umap_2, y_train)


In [ ]:
y_pred_gbc_umap2 = gbc_umap_2.predict(x_test_umap_2)

In [ ]:
gbc_umap_2.score(x_test_umap_2, y_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred_gbc_umap2 )
disp = ConfusionMatrixDisplay(cm, display_labels = gbc_umap_2.classes_)
disp.plot()

In [ ]:
print(classification_report(y_test, y_pred_gbc_umap2, labels = gbc_umap_2.classes_))

In [ ]:
x_train_umap_2.shape

In [ ]:
#df_x_train_umap_2 = pd.DataFrame(x_train_umap_2, columns = ['umap2_1','umap2_2'])
#df_x_train_umap_2.shape

In [ ]:
#x_train = x_train.iloc[:,:-2]
x_train.shape

In [ ]:
#x_train_add_umap2 = pd.concat([x_train, df_x_train_umap_2 ], axis=1 ,ignore_index= True) #strange, extra rows???
#x_train_add_umap2 = pd.concat([x_train, df_x_train_umap_2 ], axis=1 ,ignore_index=False) #strange, extra rows???
#x_train_add_umap2 = x_train.join(df_x_train_umap_2) #strange result in NAN
x_train_add_umap2 = x_train.copy()          
x_train_add_umap2['umap2_1'] = x_train_umap_2[:,0]
x_train_add_umap2['umap2_2'] = x_train_umap_2[:,1]                           
                           
                           

In [ ]:
x_train_add_umap2.shape

In [ ]:
gbc_add_umap_2 = GradientBoostingClassifier(n_estimators = 300 , verbose = 1)
gbc_add_umap_2.fit(x_train_add_umap2, y_train)





In [ ]:
x_test_add_umap2 = x_test.copy()       
x_test_add_umap2['umap2_1'] = x_test_umap_2[:,0] 
x_test_add_umap2['umap2_2'] = x_test_umap_2[:,1]    

In [ ]:
y_pred_gbc_add_umap2 = gbc_add_umap_2.predict(x_test_add_umap2)

In [ ]:
gbc_add_umap_2.score(x_test_add_umap2, y_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred_gbc_add_umap2 )
disp = ConfusionMatrixDisplay(cm, display_labels = gbc_add_umap_2.classes_)
disp.plot()

## Note

It seems due to class imbalance, GBC always predict class_2 which gives overall good result. GBC on UMAP 2 dimension alone, shows underfit. GBC with raw data + UMAP 2 as extra feature does not give good result. Considering UMAP for reduced dimension of 6 instead of 2. 

In [ ]:
x_train.head()

In [ ]:
umap6 = UMAP(n_components = 6, random_state = 1)
x_train_umap6 = umap6.fit_transform(x_train, y_train)
x_test_umap6 = umap6.transform(x_test)

In [ ]:
gbc_umap6_only = GradientBoostingClassifier(n_estimators = 300,  verbose = 1)
gbc_umap6_only.fit(x_train_umap6, y_train)

In [ ]:
y_pred_gbc_umap6_only = gbc_umap6_only.predict(x_test_umap6)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, y_pred_gbc_umap6_only), display_labels = gbc_umap6_only.classes_)
disp.plot()

Umap6 alone is rubbish. Try add this to raw data and see. 

In [ ]:
#x_train_add_umap6 = pd.concat([x_train, pd.DataFrame(x_train_umap6, columns=['umap6_'+str(i) for i in range(1,7)])], axis = 1 , ignore_index = True)
#x_train_add_umap6.head()
x_train_add_umap6 = x_train.copy()
x_test_add_umap6 = x_test.copy()       
for i in range(6):
    yvar = 'umap6_'+str(i+1)
    x_train_add_umap6[yvar] = x_train_umap6[:,i]
    x_test_add_umap6[yvar] = x_test_umap6[:,i]
x_train_add_umap6.head()

In [ ]:
x_train_add_umap6.shape, x_test_add_umap6.shape

In [ ]:
gbc_add_umap_6 = GradientBoostingClassifier(n_estimators = 300 , verbose = 1)
gbc_add_umap_6.fit(x_train_add_umap6, y_train)

In [ ]:
y_pred_gbc_umap6 = gbc_add_umap_6.predict(x_test_add_umap6)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, y_pred_gbc_umap6), display_labels = gbc_add_umap_6.classes_)
disp.plot()

In [ ]:
x_train.head()

## test component of 3 and 4

In [ ]:
umap_only = {}
raw_add_umap = {}
for n_components in range(3,5):
    print('%'*30)
    print(f'Umap n_components = {n_components}')
    print(f'Only {n_components} Umap components as feature for classification')
    umap_only[n_components] = UMAP(n_components = n_components, random_state = 1)
    umap_only['x_train_'+str(n_components)] = umap_only[n_components].fit_transform(x_train, y_train)
    umap_only['x_test_'+str(n_components)] = umap_only[n_components].transform(x_test)
    umap_only['gbc_umap'+str(n_components)] = GradientBoostingClassifier(n_estimators = 300, verbose = 1)
    umap_only['gbc_umap'+str(n_components)].fit(umap_only['x_train_'+str(n_components)], y_train)
    umap_only['y_pred_gbc_umap'+str(n_components)] = umap_only['gbc_umap'+str(n_components)].predict(umap_only['x_test_'+str(n_components)])
    #print(umap_only['y_pred_gbc_umap'+str(n_components)].shape, y_test.shape)
    print(umap_only['gbc_umap'+str(n_components)].score( umap_only['x_test_'+str(n_components)] , y_test))
    disp = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, umap_only['y_pred_gbc_umap'+str(n_components)]), display_labels = umap_only['gbc_umap'+str(n_components)].classes_)
    disp.plot()
    print(classification_report(y_test, umap_only['y_pred_gbc_umap'+str(n_components)], labels = umap_only['gbc_umap'+str(n_components)].classes_))
    
    print('='*20)
    print(f'Original Feature +  {n_components} Umap components as feature for classification')
    raw_add_umap['x_train_umap'+str(n_components)] = x_train.copy()
    raw_add_umap['x_test_umap'+str(n_components)]  = x_test.copy()       
    for i in range(n_components):
        yvar = 'umap'+str(n_components)+'_'+str(i+1)
        raw_add_umap['x_train_umap'+str(n_components)][yvar] = umap_only['x_train_'+str(n_components)][:,i]
        raw_add_umap['x_test_umap'+str(n_components)][yvar] = umap_only['x_test_'+str(n_components)][:,i]
    raw_add_umap['gbc_umap'+str(n_components)] = GradientBoostingClassifier(n_estimators = 300, verbose = 1)
    raw_add_umap['gbc_umap'+str(n_components)].fit(raw_add_umap['x_train_umap'+str(n_components)], y_train)
    raw_add_umap['y_pred_gbc_umap'+str(n_components)] = raw_add_umap['gbc_umap'+str(n_components)].predict(raw_add_umap['x_test_umap'+str(n_components)])
    print(raw_add_umap['gbc_umap'+str(n_components)].score( raw_add_umap['x_test_umap'+str(n_components)] , y_test))
    disp = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(y_test, raw_add_umap['y_pred_gbc_umap'+str(n_components)]), display_labels = raw_add_umap['gbc_umap'+str(n_components)].classes_)
    disp.plot()
    
    print(classification_report(y_test, raw_add_umap['y_pred_gbc_umap'+str(n_components)], labels = raw_add_umap['gbc_umap'+str(n_components)].classes_))
    